### Import Required Libraries and Set Up Environment Variables

In [143]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [145]:
# Set environment variables from the .env in the local environment
load_dotenv('local2.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(tmdb_api_key)

ENTER YOUR KEY HERE


### Access the New York Times API

In [127]:
# Build URL&
query_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name%3A%22Movies%22%20AND%20type_of_material%3A%22Review%22&sort=newest&page=0&api-key=" + nyt_api_key

In [128]:
results = requests.get(query_url)
results.text

'{"status":"OK","copyright":"Copyright (c) 2024 The New York Times Company. All Rights Reserved.","response":{"docs":[{"abstract":"The Brazilian director Kleber Mendonça Filho draws on fact and fiction in this image-rich documentary that moves fast and far, but always returns home.","web_url":"https://www.nytimes.com/2024/01/25/movies/pictures-of-ghosts-review.html","snippet":"The Brazilian director Kleber Mendonça Filho draws on fact and fiction in this image-rich documentary that moves fast and far, but always returns home.","lead_paragraph":"Early in “Pictures of Ghosts,” an exhilarating documentary about specters onscreen and off, the Brazilian director Kleber Mendonça Filho, pulls out a VHS tape. It’s of a 1981 TV interview with his mother, Joselice, a historian who died at age 54. In close-up, she discusses gathering information left out of history, an approach that her son has embraced here. After the tape abruptly cuts off, he says in voice-over, “it may seem like I’m discussin

In [129]:
reviews = []

# Loop through pages 0-19
for page in range(20):
    # Create a query with a page number
    query_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q=movie&fq=section_name%3A%22Movies%22&page={page}&api-key={nyt_api_key}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url).json()
    
    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # Loop through the reviews["response"]["docs"] and append each review to the list
        for doc in response["response"]["docs"]:
            reviews.append(doc)
        
        # Print the page that was just retrieved
        print(f"Retrieved reviews for page {page + 1}")
        
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results for page {page + 1}")
        break
reviews

Retrieved reviews for page 1


KeyboardInterrupt: 

In [202]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_results = reviews[:5]

# Format the data using json.dumps with indent=4
formatted_results = json.dumps(preview_results, indent=4)
formatted_results

'[\n    {\n        "abstract": "With \\u201cMean Girls,\\u201d \\u201cWonka\\u201d and \\u201cThe Color Purple,\\u201d why have studios spent much of their marketing budget downplaying and disguising their movie musicals?",\n        "web_url": "https://www.nytimes.com/2024/01/12/movies/mean-girls-trailer-musical.html",\n        "snippet": "With \\u201cMean Girls,\\u201d \\u201cWonka\\u201d and \\u201cThe Color Purple,\\u201d why have studios spent much of their marketing budget downplaying and disguising their movie musicals?",\n        "lead_paragraph": "Regina George has a secret. She sings.",\n        "print_section": "A",\n        "print_page": "1",\n        "source": "The New York Times",\n        "multimedia": [\n            {\n                "rank": 0,\n                "subtype": "xlarge",\n                "caption": null,\n                "credit": null,\n                "type": "image",\n                "url": "images/2024/01/11/multimedia/mean-girls-trailer1-cjvb/mean-girls-

In [212]:
df= pd.json_normalize(reviews)


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()


In [214]:
df['Title'] = df['headline.main'].apply(extract_title)

df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
0,"With “Mean Girls,” “Wonka” and “The Color Purp...",https://www.nytimes.com/2024/01/12/movies/mean...,"With “Mean Girls,” “Wonka” and “The Color Purp...",Regina George has a secret. She sings.,A,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-12T12:00:03+0000,...,Reporter’s Notebook,None,"Get In, Loser, We’re Going to Watch a Movie. I...",None,None,None,By Alexis Soloski,"[{'firstname': 'Alexis', 'middlename': None, '...",None,NaN
1,All five nominees are international features f...,https://www.nytimes.com/2024/01/26/movies/osca...,All five nominees are international features f...,Most post-nomination Oscar chatter focuses on ...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Documentary Fil...",2024-01-26T19:33:05+0000,...,Documentary Download,None,None,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,NaN
2,The jury focused largely on under-the-radar ti...,https://www.nytimes.com/2024/01/26/movies/sund...,The jury focused largely on under-the-radar ti...,"“In the Summers,” an independent film about tw...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-26T18:57:05+0000,...,The Projectionist,None,None,None,None,None,By Kyle Buchanan,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None,NaN
3,The complexities of 21st-century romantic enta...,https://www.nytimes.com/2024/01/26/movies/mutt...,The complexities of 21st-century romantic enta...,Vuk Lungulov-Klotz’s micro-budget New York dra...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-26T12:00:01+0000,...,Beyond the Algorithm,None,None,None,None,None,By Jason Bailey,"[{'firstname': 'Jason', 'middlename': None, 'l...",None,NaN
4,A documentary about Will Ferrell and his frien...,https://www.nytimes.com/2024/01/26/movies/sund...,A documentary about Will Ferrell and his frien...,"On Tuesday, after days of tramping around Park...",C,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-26T17:33:52+0000,...,Critic’s Notebook,None,A Stirring Road Trip For Lovers Of Movies,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,NaN
5,The Brazilian director Kleber Mendonça Filho d...,https://www.nytimes.com/2024/01/25/movies/pict...,The Brazilian director Kleber Mendonça Filho d...,"Early in “Pictures of Ghosts,” an exhilarating...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Documentary Fil...",2024-01-25T16:40:37+0000,...,Critic’s Pick,None,His Love for Cinema and Home Are Inseparable,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Pictures of Ghosts
6,The star and co-writer of “Hit Man” heard that...,https://www.nytimes.com/2024/01/25/movies/glen...,The star and co-writer of “Hit Man” heard that...,"Glen Powell doesn’t want for much these days, ...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-25T10:01:48+0000,...,The Projectionist,None,None,None,None,None,By Kyle Buchanan,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None,NaN
7,"The former first daughter’s short, “The Heart,...",https://www.nytimes.com/2024/01/26/movies/mali...,"The former first daughter’s short, “The Heart,...","If you’re a celebrity seeking to rebrand, the ...",C,4,The 

In [222]:

df2 = df

In [217]:
import re
def extract_title(text):
    # Use regular expression to extract text between unicode characters \u2018 and \u2019
    # and ensure it ends with " Review"
    match = re.search(r'\u2018(.*?)\u2019 Review', text)
    if match:
        return match.group(1)
    else:
        return None

In [87]:
dflist = df['title'].tolist()

In [89]:

dflist = [value for value in dflist if value is not None]
dflist

['Pictures of Ghosts',
 'The Underdoggs',
 'American Star',
 'Miller’s Girl',
 'Tótem',
 'Sometimes I Think About Dying',
 'The Peasants',
 'Mambar Pierrette',
 'Lil Nas X: Long Live Montero',
 'The Breaking Ice',
 'The End We Start From',
 'Which Brings Me to You',
 'I.S.S.',
 'The Kitchen',
 'Pasang: In the Shadow of Everest',
 'Inside the Yellow Cocoon Shell',
 'June',
 'Lift',
 'Role Play',
 'Mean Girls',
 'The Settlers',
 'Driving Madeleine',
 'Apolonia, Apolonia',
 'Inshallah a Boy',
 'The Beekeeper',
 'Night Swim',
 'Society of the Snow',
 'He Went That Way',
 'Artie Shaw: Time Is All You’ve Got',
 'Mayhem!',
 'Memory',
 'Good Grief',
 'Rose',
 'Ferrari',
 'The Boys in the Boat',
 'Occupied City',
 'The Crime is Mine',
 'The Teachers’ Lounge',
 'Aquaman and the Lost Kingdom',
 'Rebel Moon — Part One: Child of Fire',
 'Migration',
 'Freud’s Last Session',
 'Anyone but You',
 'The Inner Cage',
 'All of Us Strangers',
 'The Iron Claw',
 'The Color Purple',
 'The Family Plan',
 'The

In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [179]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string


In [90]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
dflist

['Pictures of Ghosts',
 'The Underdoggs',
 'American Star',
 'Miller’s Girl',
 'Tótem',
 'Sometimes I Think About Dying',
 'The Peasants',
 'Mambar Pierrette',
 'Lil Nas X: Long Live Montero',
 'The Breaking Ice',
 'The End We Start From',
 'Which Brings Me to You',
 'I.S.S.',
 'The Kitchen',
 'Pasang: In the Shadow of Everest',
 'Inside the Yellow Cocoon Shell',
 'June',
 'Lift',
 'Role Play',
 'Mean Girls',
 'The Settlers',
 'Driving Madeleine',
 'Apolonia, Apolonia',
 'Inshallah a Boy',
 'The Beekeeper',
 'Night Swim',
 'Society of the Snow',
 'He Went That Way',
 'Artie Shaw: Time Is All You’ve Got',
 'Mayhem!',
 'Memory',
 'Good Grief',
 'Rose',
 'Ferrari',
 'The Boys in the Boat',
 'Occupied City',
 'The Crime is Mine',
 'The Teachers’ Lounge',
 'Aquaman and the Lost Kingdom',
 'Rebel Moon — Part One: Child of Fire',
 'Migration',
 'Freud’s Last Session',
 'Anyone but You',
 'The Inner Cage',
 'All of Us Strangers',
 'The Iron Claw',
 'The Color Purple',
 'The Family Plan',
 'The

### Access The Movie Database API

In [147]:
api = '7859d53ebe6b359a09b178c430607bd1'

In [149]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + api


In [162]:
import requests

params = {
    'query': 'Jack Reacher',
    'api_key': api,
}

response = requests.get('https://api.themoviedb.org/3/search/movie', params=params)

In [163]:
response.text

'{"page":1,"results":[{"adult":false,"backdrop_path":"/gFWeOJRI3vuV7bksf4m0IhvPHfx.jpg","genre_ids":[80,18,53,28],"id":75780,"original_language":"en","original_title":"Jack Reacher","overview":"One morning in an ordinary town, five people are shot dead in a seemingly random attack. All evidence points to a single suspect: an ex-military sniper who is quickly brought into custody. The interrogation yields one written note: \'Get Jack Reacher!\'. Reacher, an enigmatic ex-Army investigator, believes the authorities have the right man but agrees to help the sniper\'s defense attorney. However, the more Reacher delves into the case, the less clear-cut it appears. So begins an extraordinary chase for the truth, pitting Jack Reacher against an unexpected enemy, with a skill for violence and a secret to keep.","popularity":90.92,"poster_path":"/uQBbjrLVsUibWxNDGA4Czzo8lwz.jpg","release_date":"2012-12-20","title":"Jack Reacher","video":false,"vote_average":6.619,"vote_count":6643},{"adult":fals

In [171]:

tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0


titles = dflist

for title in titles:
   
    if request_counter > 0 and request_counter % 50 == 0:
        print("Sleeping for 10 seconds to avoid API rate limits...")
        time.sleep(10)
    

    request_counter += 1

    # Perform a "GET" request for The Movie Database
    api_key = api
    query = title.replace(" ", "+")
    url = f"https://api.themoviedb.org/3/search/movie?query={query}&api_key={api_key}"

    try:

        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            if data.get("results") is not None and len(data.get("results")) > 0:
                first_movie = data["results"][0]

                # Extract the relevant data
                movie_info = {
                    "Title": first_movie["title"],
                    "Release Date": first_movie["release_date"],
                    "Overview": first_movie["overview"],
                    "Genres": [genre_id for genre_id in first_movie["genre_ids"]],

                }

                
                tmdb_movies_list.append(movie_info)


                print(f"Found movie: {first_movie['title']}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


for movie in tmdb_movies_list:
    print(movie)


Found movie: Pictures of Ghosts
Found movie: The Underdoggs
Found movie: American Star
Found movie: Miller's Girl
Found movie: Tótem
Found movie: Sometimes I Think About Dying
Found movie: The Peasants
Found movie: Mambar Pierrette
Found movie: Lil Nas X: Long Live Montero
Found movie: Breaking the Ice
Found movie: The End We Start From
Found movie: Which Brings Me to You
Found movie: I.S.S.
Found movie: The Kitchen
Found movie: Pasang: In the Shadow of Everest
Found movie: Inside the Yellow Cocoon Shell
Found movie: June
Found movie: Lift
Found movie: Role Play
Found movie: Mean Girls
Found movie: The Settlers
Found movie: Driving Madeleine
Found movie: Apolonia, Apolonia
Found movie: Inshallah a Boy
Found movie: The Beekeeper
Found movie: Night Swim
Found movie: Society of the Snow
Found movie: He Went That Way
Found movie: Artie Shaw: Time Is All You've Got
Found movie: Mayhem!
Found movie: Memory
Found movie: Good Grief
Found movie: Rose
Found movie: Ferrari
Found movie: The Boys i

In [11]:
# Create an empty list to store the results
tmdb_movies_list = []
request_counter = 1
# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
F

In [175]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_5_results = tmdb_movies_list[:5]


formatted_json = json.dumps(first_5_results, indent=4)
formatted_json

'[\n    {\n        "Title": "Pictures of Ghosts",\n        "Release Date": "2023-08-24",\n        "Overview": "Downtown Recife\\u2019s classic movie palaces from the 20th century are mostly gone. That city area is now an archaeological site of sorts that reveals aspects of life in society which have been lost. And that\\u2019s just part of the story.",\n        "Genres": [\n            99\n        ]\n    },\n    {\n        "Title": "The Underdoggs",\n        "Release Date": "2024-01-25",\n        "Overview": "Jaycen \'Two Js\' Jennings is a washed-up former pro football star who has hit rock bottom. When he is sentenced to do community service coaching the Underdoggs, an unruly pee-wee football team in his hometown, he sees it mostly as an opportunity to rebuild his public image. But in the process, he may just turn his life around and rediscover his love of the game.",\n        "Genres": [\n            35\n        ]\n    },\n    {\n        "Title": "American Star",\n        "Release D

In [177]:
# Convert the results to a DataFrame
df = pd.DataFrame(tmdb_movies_list)
df1 = df

In [225]:
df1

,Title,Release Date,Overview,Genres
0,Pictures of Ghosts,2023-08-24,Downtown Recife’s classic movie palaces from t...,[99]
1,The Underdoggs,2024-01-25,Jaycen 'Two Js' Jennings is a washed-up former...,[35]
2,American Star,2024-01-26,An assassin on a final assignment in Fuerteven...,"[53, 9648, 18]"
3,Miller's Girl,2024-01-18,A talented young writer embarks on a creative ...,"[18, 35]"
4,Tótem,2023-09-07,"In a bustling Mexican household, seven-year-ol...",[18]
...,...,...,...,...
72,Godzilla Minus One,2023-11-03,"In postwar Japan, a new terror rises. Will the...","[878, 27, 28]"
73,Who We Become,2023-05-11,"Kapwa, a Filipino term that means ""togethernes...",[99]
74,Bad Press,2023-01-22,When the Muscogee Nation suddenly begins censo...,[99]
75,Family Switch,2023-11-30,When the Walker family members switch bodies w...,"[35, 14, 10751]"


In [223]:
df2

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
0,"With “Mean Girls,” “Wonka” and “The Color Purp...",https://www.nytimes.com/2024/01/12/movies/mean...,"With “Mean Girls,” “Wonka” and “The Color Purp...",Regina George has a secret. She sings.,A,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-12T12:00:03+0000,...,Reporter’s Notebook,None,"Get In, Loser, We’re Going to Watch a Movie. I...",None,None,None,By Alexis Soloski,"[{'firstname': 'Alexis', 'middlename': None, '...",None,NaN
1,All five nominees are international features f...,https://www.nytimes.com/2024/01/26/movies/osca...,All five nominees are international features f...,Most post-nomination Oscar chatter focuses on ...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Documentary Fil...",2024-01-26T19:33:05+0000,...,Documentary Download,None,None,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,NaN
2,The jury focused largely on under-the-radar ti...,https://www.nytimes.com/2024/01/26/movies/sund...,The jury focused largely on under-the-radar ti...,"“In the Summers,” an independent film about tw...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-26T18:57:05+0000,...,The Projectionist,None,None,None,None,None,By Kyle Buchanan,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None,NaN
3,The complexities of 21st-century romantic enta...,https://www.nytimes.com/2024/01/26/movies/mutt...,The complexities of 21st-century romantic enta...,Vuk Lungulov-Klotz’s micro-budget New York dra...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-26T12:00:01+0000,...,Beyond the Algorithm,None,None,None,None,None,By Jason Bailey,"[{'firstname': 'Jason', 'middlename': None, 'l...",None,NaN
4,A documentary about Will Ferrell and his frien...,https://www.nytimes.com/2024/01/26/movies/sund...,A documentary about Will Ferrell and his frien...,"On Tuesday, after days of tramping around Park...",C,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-26T17:33:52+0000,...,Critic’s Notebook,None,A Stirring Road Trip For Lovers Of Movies,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,NaN
5,The Brazilian director Kleber Mendonça Filho d...,https://www.nytimes.com/2024/01/25/movies/pict...,The Brazilian director Kleber Mendonça Filho d...,"Early in “Pictures of Ghosts,” an exhilarating...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Documentary Fil...",2024-01-25T16:40:37+0000,...,Critic’s Pick,None,His Love for Cinema and Home Are Inseparable,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Pictures of Ghosts
6,The star and co-writer of “Hit Man” heard that...,https://www.nytimes.com/2024/01/25/movies/glen...,The star and co-writer of “Hit Man” heard that...,"Glen Powell doesn’t want for much these days, ...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-01-25T10:01:48+0000,...,The Projectionist,None,None,None,None,None,By Kyle Buchanan,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None,NaN
7,"The former first daughter’s short, “The Heart,...",https://www.nytimes.com/2024/01/26/movies/mali...,"The former first daughter’s short, “The Heart,...","If you’re a celebrity seeking to rebrand, the ...",C,4,The 

### Merge and Clean the Data for Export

In [224]:
merged_df = pd.merge(df1, df2, on="Title", how="inner")
merged_df

,Title,Release Date,Overview,Genres,abstract,web_url,snippet,lead_paragraph,print_section,print_page,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Pictures of Ghosts,2023-08-24,Downtown Recife’s classic movie palaces from t...,[99],The Brazilian director Kleber Mendonça Filho d...,https://www.nytimes.com/2024/01/25/movies/pict...,The Brazilian director Kleber Mendonça Filho d...,"Early in “Pictures of Ghosts,” an exhilarating...",C,9,...,‘Pictures of Ghosts’ Review: Layers of Love an...,Critic’s Pick,None,His Love for Cinema and Home Are Inseparable,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
1,The Underdoggs,2024-01-25,Jaycen 'Two Js' Jennings is a washed-up former...,[35],"Snoop Dogg, as an egomaniacal retired football...",https://www.nytimes.com/2024/01/25/movies/unde...,"Snoop Dogg, as an egomaniacal retired football...",It’s not until the film’s postscript that “The...,C,10,...,‘The Underdoggs’ Review: Guess Who the New Coa...,None,None,The Underdoggs,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None


In [226]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df1, df2, on="Title", how="inner")
merged_df

,Title,Release Date,Overview,Genres,abstract,web_url,snippet,lead_paragraph,print_section,print_page,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Pictures of Ghosts,2023-08-24,Downtown Recife’s classic movie palaces from t...,[99],The Brazilian director Kleber Mendonça Filho d...,https://www.nytimes.com/2024/01/25/movies/pict...,The Brazilian director Kleber Mendonça Filho d...,"Early in “Pictures of Ghosts,” an exhilarating...",C,9,...,‘Pictures of Ghosts’ Review: Layers of Love an...,Critic’s Pick,None,His Love for Cinema and Home Are Inseparable,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
1,The Underdoggs,2024-01-25,Jaycen 'Two Js' Jennings is a washed-up former...,[35],"Snoop Dogg, as an egomaniacal retired football...",https://www.nytimes.com/2024/01/25/movies/unde...,"Snoop Dogg, as an egomaniacal retired football...",It’s not until the film’s postscript that “The...,C,10,...,‘The Underdoggs’ Review: Guess Who the New Coa...,None,None,The Underdoggs,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [227]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns='byline.person')


In [233]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates(subset=['Title'])
merged_df.reset_index()

,index,Title,Release Date,Overview,Genres,abstract,web_url,snippet,lead_paragraph,print_section,...,uri,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,0,Pictures of Ghosts,2023-08-24,Downtown Recife’s classic movie palaces from t...,[99],The Brazilian director Kleber Mendonça Filho d...,https://www.nytimes.com/2024/01/25/movies/pict...,The Brazilian director Kleber Mendonça Filho d...,"Early in “Pictures of Ghosts,” an exhilarating...",C,...,nyt://article/a8387394-60fc-5413-8e55-9bb11820...,‘Pictures of Ghosts’ Review: Layers of Love an...,Critic’s Pick,None,His Love for Cinema and Home Are Inseparable,None,None,None,By Manohla Dargis,None
1,1,The Underdoggs,2024-01-25,Jaycen 'Two Js' Jennings is a washed-up former...,[35],"Snoop Dogg, as an egomaniacal retired football...",https://www.nytimes.com/2024/01/25/movies/unde...,"Snoop Dogg, as an egomaniacal retired football...",It’s not until the film’s postscript that “The...,C,...,nyt://article/dbcf36ef-f1c0-560d-b5e4-0327f083...,‘The Underdoggs’ Review: Guess Who the New Coa...,None,None,The Underdoggs,None,None,None,By Brandon Yu,None


In [234]:
# Export data to CSV without the index
merged_df.to_csv('merged_movies.csv', index=False)